In [22]:
from tensorflow import keras
import numpy as np
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [19]:
data = np.arange(1*3*1).reshape(1,3,1)
data = data+1
print(data, data.shape)

# 한개의 layer에 한개의 Unit만 적용된 것이다. 데이터 아직 input X
m = keras.Sequential([
    keras.layers.SimpleRNN(3, input_shape=(None, 1), activation=None,
                          kernel_initializer = keras.initializers.Constant(0.5),
                          recurrent_initializer =  keras.initializers.Ones())
])

print(m.layers[0].weights)
m.summary()
#m.predict(data) # RNN 계산

[[[1]
  [2]
  [3]]] (1, 3, 1)
[<tf.Variable 'simple_rnn_11/simple_rnn_cell_11/kernel:0' shape=(1, 3) dtype=float32, numpy=array([[0.5, 0.5, 0.5]], dtype=float32)>, <tf.Variable 'simple_rnn_11/simple_rnn_cell_11/recurrent_kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>, <tf.Variable 'simple_rnn_11/simple_rnn_cell_11/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_11 (SimpleRNN)   (None, 3)                 15        
                                                                 
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [ ]:
data = np.arange(1*3*1).reshape(1,3,1)
data = data+1
print(data, data.shape)

# 한개의 layer에 한개의 Unit만 적용된 것이다. 데이터 아직 input X
m = keras.Sequential([
    keras.layers.LSTM(1, input_shape=(None, 1), activation=None,
                          kernel_initializer = keras.initializers.Constant(0.5),
                          recurrent_initializer =  keras.initializers.Ones())
])

print(m.layers[0].weights)
m.summary()
#m.predict(data) # RNN 계산

In [21]:
data = np.arange(1*3*1).reshape(1,3,1)
data = data+1
print(data, data.shape)

# 한개의 layer에 한개의 Unit만 적용된 것이다. 데이터 아직 input X
m = keras.Sequential([
    keras.layers.GRU(1, input_shape=(None, 1), activation=None,
                          kernel_initializer = keras.initializers.Constant(0.5),
                          recurrent_initializer =  keras.initializers.Ones())
])

print(m.layers[0].weights)
m.summary()
#m.predict(data) # RNN 계산

[[[1]
  [2]
  [3]]] (1, 3, 1)
[<tf.Variable 'gru/gru_cell/kernel:0' shape=(1, 3) dtype=float32, numpy=array([[0.5, 0.5, 0.5]], dtype=float32)>, <tf.Variable 'gru/gru_cell/recurrent_kernel:0' shape=(1, 3) dtype=float32, numpy=array([[1., 1., 1.]], dtype=float32)>, <tf.Variable 'gru/gru_cell/bias:0' shape=(2, 3) dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>]
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1)                 12        
                                                                 
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Attention layer
#Sequential 모델이 아니라 Functional 모델이다.

query = tf.keras.layers.Input(shape=(None,3,))
value = tf.keras.layers.Input(shape=(4,2,)) 
key = tf.keras.layers.Input(shape=(4,3,))

x = tf.keras.layers.Attention()([query, value, key])
model = tf.keras.models.Model(inputs=[query, value, key], outputs=x)

temp_k = tf.constant([[[10,0,0],
                       [0,10,0],
                       [0,0,10],
                       [0,0,10]]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[[   1,0],
                       [  10,0],
                       [ 100,5],
                       [1000,6]]], dtype=tf.float32)  # (4, 2)

temp_q = tf.constant([[[0, 10, 0]]], dtype=tf.float32)  # (1, 3)

model.predict([temp_q,temp_v,temp_k])

1/1 [==============================] - 0s 82ms/step


2023-06-21 20:28:47.880872: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


array([[[10.,  0.]]], dtype=float32)

In [33]:
# Attention layer의 연산과정이다. 

# Key하고 query값을 곱해서 
print('1단계')
print(temp_q.shape)
print(temp_k.shape)

scores = tf.matmul(temp_q, temp_k, transpose_b = True)
#matrics multi
print(scores.shape, scores)

print('')

# 그 곱한 값을 softmax값을 통과시키고
print('2단계')
dist = tf.nn.softmax(scores)
print(dist.shape, dist)

print('')

# 그 softmax통과한 값을 value와 더하는 것이 attention 연산이 되는 과정이다.
print('3단계')
out = tf.matmul(dist, temp_v)
print(out.shape, out)

1단계
(1, 1, 3)
(1, 4, 3)
(1, 1, 4) tf.Tensor([[[  0. 100.   0.   0.]]], shape=(1, 1, 4), dtype=float32)

2단계
(1, 1, 4) tf.Tensor([[[0. 1. 0. 0.]]], shape=(1, 1, 4), dtype=float32)

3단계
(1, 1, 2) tf.Tensor([[[10.  0.]]], shape=(1, 1, 2), dtype=float32)


![nn](1_wT3X_UAZ6MAI_hdqejUVMQ.webp)